In [2]:
import jinja2
import json
import os

def render_template(template_file, context):
    loader = jinja2.FileSystemLoader(searchpath=".")
    env = jinja2.Environment(loader=loader)
    template = env.get_template(template_file)
    rendered_text = template.render(context)
    return rendered_text

with open('./../program-data.json') as f:
    schedule = json.load(f)

doc = '''# Tech Needs: SuperCollider Symposium 2025

Thursday, March 13 - Saturday, March 15, 2025

Presented here in _sound check_ order, not concert order.

'''

installations_doc = '\n\n# Installations\n\nSuperCollider Symposium 2025\n\n---\n\n'

workshops_doc = '\n\n# Workshops\n\nSuperCollider Symposium 2025\n\n'

for day in schedule['days']:
    doc += f'\n\n# {day['day']}\n\n'
    
    if 'installations' in day:
        for installation in day['installations']:
            installation['day'] = day['day']
            installations_doc += render_template('installation.jinja.md', installation)
    
    for event in day['events']:
        
        if event['type'] == 'concert':
            doc += f'\n\n# {event["title"]}\n\n---\n\n'
            duration = 0
            for work in event['works']:
                duration += work['duration']
            doc += f'Total duration: {duration} minutes\n\n'
            
            reversed_works = event['works'][::-1]
            for work in reversed_works:
                doc += render_template('work.jinja.md', work)
        
        if event['type'] == 'workshop':
            workshops_doc += f'\n\n---\n\n# {event["title"]}\n\n{day['day']}\n\n{event['time']}\n\nBoth workshops are one hour and occur simultaneously.\n\n'
            for workshop in event['workshops']:
                workshops_doc += render_template('workshop.jinja.md', workshop)
                
        if event['type'] == 'installations':
            for installation in event['installations']:
                installation['day'] = day['day']
                installations_doc += render_template('installation.jinja.md', installation)

def create_pdf(extensionless_path,document_string):    
    with open(f'{extensionless_path}.md', 'w') as f:
        f.write(document_string)
    os.system(f'pandoc {extensionless_path}.md -o ../{extensionless_path}.pdf')
    os.system(f'rm {extensionless_path}.md')

create_pdf('tech-needs-music',doc)
create_pdf('tech-needs-installations',installations_doc)
create_pdf('tech-needs-workshops',workshops_doc)